In [ ]:
import pandas as pd
import re
from datetime import datetime
file_path = 'payment_08-Mar-2025.xlsx'
df = pd.read_excel(file_path)
df1 = df[['UID', 'BudgetItem', 'Department', 'Amount', 'Vendor/Transfer to department', 'AmountRemarks']]
df2 = df1.rename(columns={'UID': 'Entry No', 'BudgetItem': 'Dr Ledger Name', 'Department':'Dr Cost Center', 'Amount':'Dr Amt', 'Vendor/Transfer to department':'Cr Ledger Name' })
def extract_reference(text):
    bill_match = re.search(r'Bill No\.\s*(\S+)', text)
    invoice_match = re.search(r'Invoice No\.\s*(\S+)', text)
    quotation_match = re.search(r'Quotation Ref No\.\s*(\S+)', text)
    
    if bill_match:
        return bill_match.group(1)
    elif invoice_match:
        return invoice_match.group(1)
    elif quotation_match:
        return quotation_match.group(1)
    
    return None

# Apply the function to your data frame
df2['Bill Ref No.'] = df2['AmountRemarks'].apply(extract_reference)
df2['Vch Narration'] = 'UID No. ' + df2['Entry No'].astype(str) + ' ' + df2['AmountRemarks']
df3=df2[['Entry No', 'Dr Ledger Name',	'Dr Cost Center', 'Dr Amt', 'Cr Ledger Name', 'Bill Ref No.', 'Vch Narration' ]]
df3.insert(0, 'Date', '')  # Add 'Date' at the first position
df3.insert(2, 'Vch Name', '')  # Add 'Vch No' at the third position
df3.insert(7, 'Cr Cost Center', '')  # Add 'cost center' at the eighth position
df3.insert(8, 'Cr Amt', '')  # Add 'cost center' at the ninth position
# Display the updated dataframe
df3.head(5)
df4 = df3[['Date','Entry No','Vch Name','Dr Ledger Name','Dr Amt','Dr Cost Center','Cr Ledger Name','Cr Amt','Cr Cost Center','Vch Narration','Bill Ref No.']]
df4.head(5)
# Ensure 'Dr Amt' is a string and remove commas
df3['Dr Amt'] = df3['Dr Amt'].astype(str).str.replace(',', '', regex=True)
# Convert 'Dr Amt' to numeric
df3['Dr Amt'] = pd.to_numeric(df3['Dr Amt'])
# Create a copy of df3 to df4
df4 = df3.copy()
# Calculate cumulative sum for each 'Entry No'
df4['Cr Amt'] = df3.groupby('Entry No')['Dr Amt'].transform('sum')
# Format 'Dr Amt' and 'Cr Amt' with commas and two decimal places
df4['Dr Amt'] = df4['Dr Amt'].apply(lambda x: f"{x:,.2f}")
df4['Cr Amt'] = df4['Cr Amt'].apply(lambda x: f"{x:,.2f}")
# Display the final DataFrame
df4.head(5)
tally_icm_heads = pd.read_excel('tally_icm_heads.xlsx')
tally_icm_heads = tally_icm_heads[['icm_heads','tally_heads']]
tally_icm_heads = tally_icm_heads.rename(columns={'icm_heads':'Dr Ledger Name'})
tally_icm_heads.head(5)
# Use merge instead of map to handle duplicate keys
df4 = df4.merge(tally_icm_heads, on='Dr Ledger Name', how='left')
# Drop the old 'Dr Ledger Name' column if necessary
df4.drop(columns=['Dr Ledger Name'], inplace=True)
# Rename 'tally_heads' to 'Dr Ledger Name'
df4.rename(columns={'tally_heads': 'Dr Ledger Name'}, inplace=True)
df4.head(5)
df5 = df4[['Date','Entry No','Vch Name','Dr Ledger Name','Dr Cost Center','Dr Amt','Cr Ledger Name','Cr Cost Center','Cr Amt','Bill Ref No.','Vch Narration']]
df6 = df5.drop_duplicates()
df6.tail()
# Get current date and timestamp
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# Create output file name with date and timestamp
output_filename = f'output_{current_time}.xlsx'
# Convert DataFrame to Excel
df6.to_excel(output_filename, index=False)